# Préparation

In [0]:
import numpy as np
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from contextlib import redirect_stdout

Import des annotations et des données raw :

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!cp /content/drive/My\ Drive/data_redev/coco/annotations_trainval2017.zip .
!unzip annotations_trainval2017.zip
!cp /content/drive/My\ Drive/res_eval_redev/yolov3_raw.json .
!cp /content/drive/My\ Drive/res_eval_redev/yolov3_tiny_raw.json .

Archive:  annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  


Ground truth :

In [12]:
dataDir = '.'
dataType = 'val2017'
annType = 'bbox'
annFile = '{}/annotations/instances_{}.json'.format(dataDir,dataType, annType)

cocoGt = COCO(annFile)

loading annotations into memory...
Done (t=0.67s)
creating index...
index created!


# Résultats

In [0]:
# Map category names to category ids
category_map = {cat['name']: cat['id'] for cat in cocoGt.loadCats(cocoGt.getCatIds())}
category_map['motorbike'] = category_map['motorcycle']
category_map['tvmonitor'] = category_map['tv']
category_map['pottedplant'] = category_map['potted plant']
category_map['sofa'] = category_map['couch']
category_map['diningtable'] = category_map['dining table']
category_map['aeroplane'] = category_map['airplane']


YOLOv3 :

In [0]:
with open('yolov3_raw.json', 'r') as f:
    raw = json.load(f)
outputs = raw['outputs']

# Map category names to category ids
for out in outputs:
    out["category_id"] = category_map[out["category_name"]]
    out["image_id"] = int(out["image_id"])
    del out["category_name"]

mean_inf_time = raw['mean_inf_time']
with open("yolov3_outputs_raw.json", 'w') as f:
     json.dump(outputs, f)

In [82]:
cocoDt = cocoGt.loadRes("yolov3_outputs_raw.json")

cocoEval = COCOeval(cocoGt, cocoDt, annType)
cocoEval.params.imgIds = cocoGt.getImgIds()
cocoEval.evaluate()
cocoEval.accumulate()
with open('yolov3.txt', 'w') as f:
    with redirect_stdout(f):
        cocoEval.summarize()
        print("Temps d'inférence moyen par image : ", mean_inf_time, " ms")
!cp yolov3.txt /content/drive/My\ Drive/res_eval_redev/

Loading and preparing results...
DONE (t=0.13s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=17.98s).
Accumulating evaluation results...
DONE (t=2.30s).


Tiny YOLOv3 :

In [0]:
with open('yolov3_tiny_raw.json', 'r') as f:
    raw = json.load(f)
outputs = raw['outputs']

# Map category names to category ids
for out in outputs:
    out["category_id"] = category_map[out["category_name"]]
    out["image_id"] = int(out["image_id"])
    del out["category_name"]

mean_inf_time = raw['mean_inf_time']
with open("yolov3_tiny_outputs_raw.json", 'w') as f:
     json.dump(outputs, f)

In [91]:
cocoDt = cocoGt.loadRes("yolov3_tiny_outputs_raw.json")

cocoEval = COCOeval(cocoGt, cocoDt, annType)
cocoEval.params.imgIds = cocoGt.getImgIds()
cocoEval.evaluate()
cocoEval.accumulate()
with open('yolov3_tiny.txt', 'w') as f:
    with redirect_stdout(f):
        cocoEval.summarize()
        print("Temps d'inférence moyen par image : ", mean_inf_time, " ms")
!cp yolov3_tiny.txt /content/drive/My\ Drive/res_eval_redev/

Loading and preparing results...
DONE (t=0.07s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=13.39s).
Accumulating evaluation results...
DONE (t=1.96s).


In [87]:
outputs

[{'bbox': [143.0, 378.0, 181.0, 248.0],
  'category_id': 70,
  'image_id': 581615,
  'score': 0.8752933740615845},
 {'bbox': [175.0, 189.0, 259.0, 171.0],
  'category_id': 24,
  'image_id': 31269,
  'score': 0.9791542887687683},
 {'bbox': [391.0, 162.0, 234.0, 183.0],
  'category_id': 24,
  'image_id': 31269,
  'score': 0.9652167558670044},
 {'bbox': [19.0, 235.0, 35.0, 125.0],
  'category_id': 24,
  'image_id': 31269,
  'score': 0.25303778052330017},
 {'bbox': [224.0, 93.0, 269.0, 247.0],
  'category_id': 7,
  'image_id': 60090,
  'score': 0.9395201206207275},
 {'bbox': [548.0, 193.0, 8.0, 27.0],
  'category_id': 1,
  'image_id': 60090,
  'score': 0.22401314973831177},
 {'bbox': [298.0, 87.0, 12.0, 13.0],
  'category_id': 1,
  'image_id': 45090,
  'score': 0.5717265605926514},
 {'bbox': [190.0, 95.0, 11.0, 15.0],
  'category_id': 1,
  'image_id': 45090,
  'score': 0.46206218004226685},
 {'bbox': [181.0, 93.0, 13.0, 14.0],
  'category_id': 1,
  'image_id': 45090,
  'score': 0.308411300

In [88]:
cocoGt.loadImgs([259640])

[{'coco_url': 'http://images.cocodataset.org/val2017/000000259640.jpg',
  'date_captured': '2013-11-15 05:03:30',
  'file_name': '000000259640.jpg',
  'flickr_url': 'http://farm3.staticflickr.com/2669/4126424581_7bfd3682c7_z.jpg',
  'height': 427,
  'id': 259640,
  'license': 3,
  'width': 640}]